In [1]:
%reload_kedro

                    INFO     Resolved project path as: /Users/sharkovgrigory/code/fraud_detection.  ]8;id=171290;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=736342;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#171\171]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=56825;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=858975;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    WARNING  No 'mlflow.yml' config file found in environment. Default            ]8;id=919901;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=918706;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#78\78]8;;\
                             configuration will be used. Use ``kedro mlflow init`` command in CLI                  
                             to customize the configuration.                                                       

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=181059;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=130537;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///Users/sharkovgrigory/code/fraud_detection/mlruns'                            

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=49916;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=962769;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[09/21/24 20:23:10] INFO     Kedro project fraud_detection                                          ]8;id=664761;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=399032;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=993062;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=97976;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=713394;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=629260;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [2]:
from fraud_detection.utils.row_preprocessor import get_row_preprocessor, FAST_MODEL_SETTINGS
import pandas as pd

In [3]:
train_data = catalog.load("combined_train_data")
id_cols = context.params.get("train_set_parameters").get("columns").get('id_columns')
cat_cols = ["merchant", "category"] #, 'gender', 'city', 'state', "job"]
num_cols = ["amt", 'city_pop']

train_data.head()

[09/21/24 20:23:15] INFO     Loading data from combined_train_data (CSVDataset)...              ]8;id=455934;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=978076;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/kedro/io/data_catalog.py#508\508]8;;\

,cc_num,window_id,label,merchant,category,amt,gender,city,state,city_pop,job
0,60416207185,60416207185_115,True,"fraud_Schroeder, Hauck and Treutel",entertainment,90.60,F,Fort Washakie,WY,1645,Information systems manager
1,60416207185,60416207185_115,True,"fraud_Hills, Hegmann and Schaefer",health_fitness,26.13,F,Fort Washakie,WY,1645,Information systems manager
2,60416207185,60416207185_115,True,"fraud_Langworth, Boehm and Gulgowski",shopping_net,1.24,F,Fort Washakie,WY,1645,Information systems manager
3,60416207185,60416207185_115,True,fraud_Turcotte-Halvorson,misc_pos,11.60,F,Fort Washakie,WY,1645,Information systems manager
4,60416207185,60416207185_115,True,fraud_Padberg-Sauer,home,200.83,F,Fort Washakie,WY,1645,Information systems manager


In [4]:
preprocessor = get_row_preprocessor(cat_cols, num_cols, FAST_MODEL_SETTINGS)
preprocessor.fit(train_data)

[09/21/24 20:23:17] WARNING  Each 'sentences' item should be a list of words (usually unicode       ]8;id=620885;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py\word2vec.py]8;;\:]8;id=797402;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py#558\558]8;;\
                             strings). First item here is instead plain <class 'str'>.                             

[FeatureUnion]  (step 1 of 6) Processing encoder_pipeline_merchant, total=  11.1s


[09/21/24 20:23:28] WARNING  Each 'sentences' item should be a list of words (usually unicode       ]8;id=884876;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py\word2vec.py]8;;\:]8;id=374999;file:///opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/gensim/models/word2vec.py#558\558]8;;\
                             strings). First item here is instead plain <class 'str'>.                             

[FeatureUnion]  (step 2 of 6) Processing encoder_pipeline_category, total=   8.0s
[FeatureUnion]  (step 3 of 6) Processing numeric_pipeline_amt, total=   0.0s
[FeatureUnion]  (step 4 of 6) Processing numeric_pipeline_city_pop, total=   0.0s
[FeatureUnion]  (step 5 of 6) Processing id_pipeline_window_id, total=   0.0s
[FeatureUnion]  (step 6 of 6) Processing id_pipeline_label, total=   0.0s


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 preprocessor = get_row_preprocessor(cat_cols, num_cols, FAST_MODEL_SETTINGS)                 │
│ ❱ 2 preprocessor.fit(train_data)                                                                 │
│   3                                                                                              │
│                                                                                                  │
│ /opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/sklearn/base.py:1473 in        │
│ wrapper                                                                                          │
│                                                                                                  │
│   1470 │   │   │   │   │   prefer_skip_nested_validation or global_skip_validation               │
│   1471 │   │   │   │   )                                                                         │
│   1472 │   │   │   ):                                                                            │
│ ❱ 1473 │   │   │   │   return fit_method(estimator, *args, **kwargs)                             │
│   1474 │   │                                                                                     │
│   1475 │   │   return wrapper                                                                    │
│   1476                                                                                           │
│                                                                                                  │
│ /opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/sklearn/pipeline.py:469 in fit │
│                                                                                                  │
│    466 │   │   │   Pipeline with fitted steps.                                                   │
│    467 │   │   """                                                                               │
│    468 │   │   routed_params = self._check_method_params(method="fit", props=params)             │
│ ❱  469 │   │   Xt = self._fit(X, y, routed_params)                                               │
│    470 │   │   with _print_elapsed_time("Pipeline", self._log_message(len(self.steps) - 1)):     │
│    471 │   │   │   if self._final_estimator != "passthrough":                                    │
│    472 │   │   │   │   last_step_params = routed_params[self.steps[-1][0]]                       │
│                                                                                                  │
│ /opt/miniconda3/envs/fraud_detection/lib/python3.11/site-packages/sklearn/pipeline.py:406 in     │
│ _fit                                                                                             │
│                                                                                                  │
│    403 │   │   │   else:                                                                         │
│    404 │   │   │   │   cloned_transformer = clone(transformer)                                   │
│    405 │   │   │   # Fit or load from cache the current transformer                              │
│ ❱  406 │   │   │   X, fitted_transformer = fit_transform_one_cached(                             │
│    407 │   │   │   │   cloned_transformer,                                                       │
│    408 │   │   │   │   X,                                                                        │
│    409 │   │   │   │   y,                                                                        │
│                                                                                                  │
│ /opt/miniconda3/envs/fraud_detection/lib/python3.11/site-pa

In [ ]:
transformed_data = preprocessor.transform(train_data.loc[:1,:])
transformed_data

In [ ]:
transformed_data.columns

In [ ]:
transformed_data.iloc[0,:]